In [18]:
from datasets import load_dataset
from tqdm import tqdm
import pickle

language = "Java"
license = "apache-2.0"
size = 1000


ds = load_dataset("codeparrot/github-code", languages=[language], licenses=[license], streaming=True, split="train")
ds = ds.shuffle(buffer_size=size)
ds = ds.take(size)
evaluation_dataset = []

for item in tqdm(ds, total=size):
    evaluation_dataset.append(item['code'])


eval_data_name = f"evaluation_data_{language}_{license}_{size}.pkl"

with open(eval_data_name, 'wb') as f:
    pickle.dump(evaluation_dataset, f)








100%|██████████| 1000/1000 [00:15<00:00, 63.39it/s]


In [19]:
from datasets import load_dataset
from tqdm import tqdm
import pickle

language = "Python"
license = "apache-2.0"
size = 1000


ds = load_dataset("codeparrot/github-code", languages=[language], licenses=[license], streaming=True, split="train")
ds = ds.shuffle(buffer_size=size)
ds = ds.take(size)
evaluation_dataset = []

for item in tqdm(ds, total=size):
    evaluation_dataset.append(item['code'])


eval_data_name = f"evaluation_data_{language}_{license}_{size}.pkl"

with open(eval_data_name, 'wb') as f:
    pickle.dump(evaluation_dataset, f)








100%|██████████| 1000/1000 [01:20<00:00, 12.39it/s]


In [20]:
from transformers import AutoTokenizer, AutoModelWithLMHead
import torch

def get_perplexity(model, tokenizer, input, stride=512, device='cuda'):
    encodings = tokenizer(input, return_tensors="pt")

    model.to(device)
    encodings.to(device)
    
    max_length = model.config.n_positions

    seq_len = encodings.input_ids.size(1)

    nlls = []
    prev_end_loc = 0
    for begin_loc in tqdm(range(0, seq_len, stride)):
        end_loc = min(begin_loc + max_length, seq_len)
        trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
        input_ids = encodings.input_ids[:, begin_loc:end_loc].to(device)
        target_ids = input_ids.clone()
        target_ids[:, :-trg_len] = -100

        with torch.no_grad():
            outputs = model(input_ids, labels=target_ids)
            neg_log_likelihood = outputs.loss

        nlls.append(neg_log_likelihood)

        prev_end_loc = end_loc
        if end_loc == seq_len:
            break

    ppl = torch.exp(torch.stack(nlls).mean())

    return ppl


In [21]:
model_name = "Salesforce/codegen-350M-mono"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelWithLMHead.from_pretrained(model_name)

c:\Users\Edin\anaconda3\envs\amazon\lib\site-packages\transformers\models\auto\modeling_auto.py:1295: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [28]:
sample = evaluation_dataset[0]

ppl = get_perplexity(model, tokenizer, evaluation_dataset[0])

  0%|          | 0/1 [00:00<?, ?it/s]


In [29]:
ppl

tensor(3.0420, device='cuda:0')